## Notebook

Add julia functions by calling

In [2]:
include("Julia_functions/get_room.jl")


get_room (generic function with 1 method)

In a julia code env

In [3]:
using Gridap, GridapMakie, CairoMakie, FileIO, GLMakie

In [ ]:
domain = (-1, 1, -1, 1)
partition = nx, ny = 50,50
model = CartesianDiscreteModel(domain, partition) |> simplexify

Ω = Triangulation(model)                                # make Triangulation
# fig = plot(Ω)
# wireframe!(Ω, color=:black)                             # Plot edges of each element
# scatter!(Ω, marker=:circle, markersize=20, color=:blue) # Plot mesh nodes
# display(fig)                                            # show plot


order = 1
reffe = ReferenceFE(lagrangian,Float64,order)
Testspace = TestFESpace(model,reffe,conformity=:H1)     # no Dirichlet (/ everything Neumann)

Trialspace = TrialFESpace(Testspace)                    # make trial space based on test space

degree = 2*order                                        # degree of the method used for approximating integrals over Ω                   
dΩ = Measure(Ω,degree)                                  # make the measure dΩ

Γ = BoundaryTriangulation(model)                        # triangulate the boundary ∂Ω
dΓ = Measure(Γ,degree)                                  # measure on Γ


# Definer indicator 
function χ(x, a, b)
    if a < x && x < b
        return 1
    else
        return 0
    end
end

# Varmekilden q er 1 i dette området og 0 ellers
Q(t) = χ(t, 0, 0.03)*100
q_pos(x) = χ(x[1], -0.05, 0.05) * χ(x[2], -0.10, 0.10)


τ = 0.01
k = 1
h = 0.01
T_old = 290
T_out = 290


# To store the solution over time for animation
solutions = []


for step in 1:10
    
    a(T,ϕ) = ∫( T/τ*ϕ + k*∇(ϕ)⊙∇(T))*dΩ - ∫(k*h*T*ϕ)*dΓ
    l(ϕ) = ∫(q_pos*ϕ*Q(step*τ) + T_old/τ*ϕ)*dΩ - ∫( k*h*T_out*ϕ)*dΓ 

    op = AffineFEOperator(a,l,Trialspace,Testspace)

    uh = solve(op) 
    
    # Store the solution
    push!(solutions, uh)

    T_old = uh

    fig, _ , plt = plot(Ω, uh, colormap=:plasma)            # make plot of u_h
    #wireframe!(Ω, color=:black, linewidth=1)                # add triangulation to plot
    Colorbar(fig[1,2], plt)                                 # add color bar
    display(fig)
end

# Set up the figure for the animation
# fig, ax, plt = plot(Ω, solutions[1], colormap=:plasma)

#Wireframe and colorbar for visualization
#wireframe!(Ω, color=:black, linewidth=1)
# Colorbar(fig[1, 2], plt)

# # Create the animation
# framerate = 30
# timestamps = 1:length(solutions)
# record(fig, "animation.gif", timestamps; framerate=framerate) do t_idx
#     # Update the solution plot for each time step
#     replace!(plt[1], solutions[t_idx]) # Update the plot with the current solution
# end

MethodError: MethodError: no method matching replace!(::Observable{Gridap.Geometry.BodyFittedTriangulation{2, 2, Gridap.Geometry.UnstructuredDiscreteModel{2, 2, Float64, Gridap.Geometry.Oriented}, Gridap.Geometry.UnstructuredGrid{2, 2, Float64, Gridap.Geometry.Oriented, Nothing}, Gridap.Arrays.IdentityVector{Int64}, Gridap.Geometry.Injective}}, ::Gridap.FESpaces.SingleFieldFEFunction{Gridap.CellData.GenericCellField{ReferenceDomain}})
The function `replace!` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  replace!(::Any, !Matched::Pair...; count)
   @ Base set.jl:735
  replace!(!Matched::Union{Function, Type}, ::Any; count)
   @ Base set.jl:777
  replace!(!Matched::Attributes, ::Any, !Matched::Union{AbstractScene, MakieCore.ScenePlot}, !Matched::Attributes)
   @ Makie C:\Users\Bruker\.julia\packages\Makie\Y3ABD\src\utilities\utilities.jl:307
  ...
